In [ ]:
install.packages("reshape2")
library(reshape2)
library(tidyr)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘plyr’, ‘Rcpp’



Attaching package: ‘tidyr’


The following object is masked from ‘package:reshape2’:

    smiths



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
#open the r notebook on colab
#load a formated file with two columns, first one is well number, second is the slope value, give it header where the second column name is the name of the dataset
#make a folder called DATA
dir.create("/DATA")
# set the file paths for the original and new directories
data_dir <- "/DATA/"

Warning message in dir.create("/DATA"):
“'/DATA' already exists”


In [5]:
data_dir <- "/DATA/"
# Get a list of all the files in the directory
file_list <- list.files(path = data_dir)
# Loop through the file list and read in only the TSV files
for (file_name in file_list) {
  if (endsWith(file_name, ".tsv")) {  # Check if the file ends with ".tsv"

data <- read.table(paste0(data_dir, file_name), header = TRUE, stringsAsFactors = FALSE, sep = "\t")

head(data)

#retrieve the NAME OF THE FILE INSTEAD OF THE header for later (to name the output files)
#col_header <- names(data)[2]
col_header <- file_name
# Rename the columns
colnames(data) <- c("position", "intensity")

# Split the position column into two columns
data <- separate(data, position, c("letter", "number"), sep = 1)

# Convert the number column to numeric
data$number <- as.numeric(data$number)

# Reshape the data into the desired format
data <- spread(data, letter, intensity)

# Set the row names to the number column
row.names(data) <- data$number

# Remove the number column
data <- select(data, -number)

# transpose the dataframe
data <- t(data)

# View the resulting data frame
View(data)

#every dataset was laid out by column and row, the way it looked like on an actual, physical plate, then the slope values were normalized first by substracting the column median for each value of the rexpective column, and then by substracting the row median for each value of the respective row.
#The first two columns were not used in the basic normalization procedure, as they are a separate Gaussian-distributed population. They were thus normalized separately.

data_wo_first_col <- data[, 3:ncol(data)]

# calculate median of each column using apply function
col_medians <- apply(data_wo_first_col, 2, median)
# subtract column median from all elements
df_col_centered <- sweep(data_wo_first_col, 2, col_medians, "-")
# calculate median of each row using apply function
row_medians <- apply(df_col_centered, 1, median)
# subtract row median from all elements
df_col_and_row_centered <- sweep(df_col_centered, 1, row_medians, "-")

sd(as.matrix(data_wo_first_col))
sd(as.matrix(df_col_and_row_centered))

# standardize the data frame - The resulting dataframe was standardized, meaning every value was recalculated as a distance fromp the mean, using "sd" function in R.
sd_after_centering <- sd(as.matrix(df_col_and_row_centered))

# The distance between the Guassian of the first two columns in the raw dataset versus the Gaussian of the rest of the columns dataset was then calculated and this distance was subtracted from the raw values of the first two columns. This tranformed two columns were now joined with the rest of the standardized dataset into a whole.
mean_of_col_medians <- mean(col_medians)
merged_df <- cbind(data[, 1:2] - mean_of_col_medians, df_col_and_row_centered)
df_standardized <- merged_df / sd_after_centering

ncol(data)
ncol(data_wo_first_col)
ncol(df_col_and_row_centered)
ncol(df_standardized)

# convert the standard table to long format
my_table_long <- melt(df_standardized, varnames = c("row", "col"))

# create a new data frame with row and column coordinates and values
my_new_table <- data.frame(
  coordinates = paste(my_table_long$row, my_table_long$col, sep = ""),
  values = my_table_long$value
)
write.table(my_new_table, paste0("/DATA/",col_header,"_median-corrected_long_table1.tsv"), sep="\t", quote=FALSE,)




hist(as.matrix(data), nclass = 75, main="before normalization")
hist(as.matrix(df_col_and_row_centered), nclass = 75, main="after normalization")
hist(as.matrix(df_standardized), nclass = 75, main="after standardization")

# Plot the density
plot(density(as.matrix(df_standardized) + median(as.matrix(data))),
  main="Data distribution before and after normalization",
  col = "green",
  ylim = c(0,3)
)

lines(density(as.matrix(df_col_and_row_centered) + median(as.matrix(data))), col = "blue")
lines(density(as.matrix(data)), col = "red")

# Add a legend
legend("topleft", legend = c("before normalization", "after normalization","after standardization"), col = c("red", "blue","green"), lty = 1)

#here we are writing the normalized dataset into a table in a tsv format
write.table(df_standardized, paste0("/DATA/",col_header,"_median-corrected_table1.tsv"), sep="\t", quote=FALSE,)

# Open a PDF device
pdf(paste0("/DATA/",col_header,"histogram_plots.pdf"))


hist(as.matrix(data), nclass = 75, main="before normalization")
hist(as.matrix(df_col_and_row_centered), nclass = 75, main="after normalization")
hist(as.matrix(df_standardized), nclass = 75, main="after standardization")

# Plot the density
plot(density(as.matrix(df_standardized) + median(as.matrix(data))),
  main="Data distribution before and after normalization",
  col = "green",
  ylim = c(0,3)
)

lines(density(as.matrix(df_col_and_row_centered) + median(as.matrix(data))), col = "blue")
lines(density(as.matrix(data)), col = "red")

# Add a legend
legend("topleft", legend = c("before normalization", "after normalization","after standardization"), col = c("red", "blue","green"), lty = 1)
# Close the PDF device
dev.off()

  }
}

Merge long table files

In [ ]:
# Set the working directory to the folder containing the .tsv files
setwd(data_dir)
# List all .tsv files in the folder that contain "long" in the file name
long_table_files <- list.files(pattern = "long.*\\.tsv$")

# Create an empty data frame to store the merged data
long_table_merged <- data.frame()

# Loop over each file and merge it into the merged data frame
for (file in long_table_files) {
  # Read the data from the file
  long_table_data <- read.table(file, header = TRUE, sep = "\t")

  # Add a column to the data with the name of the file
  long_table_data$filename <- basename(file)

  # Merge the data into the merged data frame
  long_table_merged <- rbind(long_table_merged, long_table_data)
}
long_table_merged <- data.frame(lapply(long_table_merged, function(x) gsub("_median-corrected_long_table1.tsv", "", x)))
# Write the merged data frame to a file called "long_merged.tsv"
write.table(long_table_merged, file = "long_tables_merged.tsv", sep = "\t", row.names = FALSE)


In [ ]:
unlink("/DATA/", recursive=TRUE)

In [ ]:
# Open a PDF device
pdf("/DATA/box_plots_and_pvals.pdf")
# one boxplot for the whole dataset before and after normalization
boxplot(as.vector(as.matrix(data)))
boxplot(as.vector(as.matrix(df_standardized)))
#use z-score on the y axis instead for the boxplot
all_values_as_vec <- as.vector(as.matrix(df_standardized))
z_scores <- scale(all_values_as_vec)
boxplot(z_scores)

#one boxplot for each column for the whole dataset before and after normalization
boxplot((as.matrix(data)))
boxplot((as.matrix(df_standardized)))


# Compute the p-values of the z-scores
p_values <- 2 * (1 - pnorm(abs(z_scores)))
final_df <- data.frame(values = all_values_as_vec, z_scores = scale(all_values_as_vec), p_values = p_values)
sorted_df <- final_df %>%
  arrange(p_values)
  View(sorted_df)
  df_standardized_mat <- as.matrix(df_standardized)
mean_x <- mean(df_standardized_mat)
sd_x <- sd(df_standardized_mat)
z_scored_matrix <- (df_standardized_mat - mean_x) / sd_x

#z_scored_matrix <- scale(as.matrix(df_standardized))

pvalues_matrix <- 2 * (1 - pnorm(abs(z_scored_matrix)))
#pvalues_vec <- 2 * (1 - pnorm(abs(as.vector(z_scored_matrix))))
#pvalues_matrix <- matrix(p_values, nrow = nrow(df_standardized_mat), ncol = ncol(df_standardized_mat))

# set the cutoff
zscore_cutoff <- -3
pvalue_cutoff <- 0.01

# print the values
# Create a data frame containing the indices and z-scores
#indices <- which(z_scored_matrix < zscore_cutoff, arr.ind = TRUE)
indices <- which(pvalues_matrix < pvalue_cutoff, arr.ind = TRUE)

#here we are making a shortlist of the best hits over the indicated threshold
df <- data.frame(
  col_index = colnames(df_standardized)[indices[,2]],
  row_index = rownames(df_standardized)[indices[,1]],
  initial_values = data[indices],
  normalized_values = df_standardized[indices],
  zscores = z_scored_matrix[indices],
  p_values = pvalues_matrix[indices]
)

sorted_df2 <- df %>%
  arrange(p_values)

View(sorted_df2)
# Close the PDF device
dev.off()
